# Multi Cloud Service Networking with Consul - Base

In [ ]:
This notebooks covers setting up your environment for the Multi-Cloud Demo.

## Prerequisites

* Azure Client
* AWS Client

Get your bearings.

In [ ]:
export MAIN_DIR=$(pwd)
export GIT_REPO_DIR=config/consul/field-workshops-consul
export WORK_DIR=$MAIN_DIR/$GIT_REPO_DIR
printf "$MAIN_DIR\n$GIT_REPO_DIR\n$WORK_DIR"

### Cloud Credentials

In [ ]:
#debugging
echo $AZURE_CREDS_FILE
echo $INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_SUBSCRIPTION_ID
echo $INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_SPN_ID
echo $INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_SPN_PASSWORD

In [ ]:
export AZURE_CREDS_FILE=${HOME}/.Azure/creds2.txt
export INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_SUBSCRIPTION_ID="REPLACEME"
export INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_SPN_ID=$(jq -r .appId ${AZURE_CREDS_FILE})
export INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_SPN_PASSWORD=$(jq -r .password ${AZURE_CREDS_FILE})

In [ ]:
export GOOGLE_CREDENTIALS=$(echo $INSTRUQT_GCP_PROJECT_CONSUL_SERVICE_ACCOUNT_KEY | base64 -d)

# echo 'export GOOGLE_CREDENTIALS=$(echo $INSTRUQT_GCP_PROJECT_CONSUL_SERVICE_ACCOUNT_KEY | base64 -d)' >> ${HOME}/.bashrc
# source ${HOME}/.bashrc

### Install Cloud Clients

In [ ]:
brew update
brew install awscli
brew install azure-cli

Login to Azure CLI

In [ ]:
az login
# az login \
#   --username "${INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_USERNAME}" \
#   --password "${INSTRUQT_AZURE_SUBSCRIPTION_CONSUL_PASSWORD}"

### AWS Setup

In [ ]:
aws configure set default.region us-east-1
aws ec2 create-default-vpc

### Azure Packer Setup

In [ ]:
POC_NAME=pphan
az group create -l westus2 -n ${POC_NAME}-packer

### jwt validation

In [ ]:
# apt install npm -y
# npm install -g jwt-cli

### Clone Assets

In [ ]:
git clone https://github.com/hashicorp/field-workshops-consul.git $GIT_REPO_DIR

In [ ]:
cp -r $GIT_REPO_DIR/instruqt-tracks/multi-cloud-service-networking-with-consul/assets/terraform .
cp -r $GIT_REPO_DIR/instruqt-tracks/multi-cloud-service-networking-with-consul/assets/packer .
cp -r $GIT_REPO_DIR/instruqt-tracks/multi-cloud-service-networking-with-consul/assets/apps .
cp -r $GIT_REPO_DIR/instruqt-tracks/multi-cloud-service-networking-with-consul/assets/policies .
cp -r $GIT_REPO_DIR/instruqt-tracks/multi-cloud-service-networking-with-consul/assets/helm .

Open folder in VS Code.

In [ ]:
code .

### Packer

In [ ]:
echo "Provisoning Packer Builds..."
# cd $MAIN_DIR/packer
pushd packer
nohup packer build hashistack.json 2>&1 | tee nohup.out &
popd

This can take awhile (~7 minutes). Go grab some coffee. You can check the status with the code editor.

**NOTE**: Need to fix.
* Azure account variables.
* License files.
* more

### Terraform

#### Provisoning Infra

In [ ]:
# cd ${MAIN_DIR}/terraform/infra
terraform -chdir=terraform/infra init
pubkey=$(cat ~/.ssh/id_rsa.pub)
cat > $MAIN_DIR/terraform/infra/terraform.tfvars << EOF
gcp_project_id="$(echo $INSTRUQT_GCP_PROJECT_CONSUL_PROJECT_ID)"
ssh_public_key="${pubkey}"
EOF

In [ ]:
terraform -chdir=terraform/infra apply -auto-approve 2>&1 | tee terraform/infra/nohup.out

#### Provisoning IAM

In [ ]:
#iam
echo "Provisoning IAM..."
cd $MAIN_DIR/terraform/iam
terraform init
cat << EOF > $MAIN_DIR/terraform/iam/terraform.tfvars
gcp_project_id="$(echo $INSTRUQT_GCP_PROJECT_CONSUL_PROJECT_ID)"
EOF
terraform apply -auto-approve 2>&1 | tee nohup.out

#### Provisoning GKE K8s Consul Cluster

In [ ]:
#gcp GKE secondary
echo "Provisoning GKE K8s Consul Cluster..."
cd /root/terraform/gcp-consul-secondary
terraform init
cat <<EOF > $MAIN_DIR/terraform/gcp-consul-secondary/terraform.tfvars
gcp_project_id="$(echo $INSTRUQT_GCP_PROJECT_CONSUL_PROJECT_ID)"
EOF
export GOOGLE_CREDENTIALS=$(echo $INSTRUQT_GCP_PROJECT_CONSUL_SERVICE_ACCOUNT_KEY | base64 -d)
nohup terraform apply -auto-approve 2>&1 | tee nohup.out &

#### Provisoning GKE K8s App Clusters

In [ ]:
#gcp GKE app clusters
echo "Provisoning GKE K8s App Clusters..."
cd $MAIN_DIR/terraform/k8s-scheduler-services
terraform init
cat <<EOF > $MAIN_DIR/terraform/k8s-scheduler-services/terraform.tfvars
gcp_project_id="$(echo $INSTRUQT_GCP_PROJECT_CONSUL_PROJECT_ID)"
EOF
export GOOGLE_CREDENTIALS=$(echo $INSTRUQT_GCP_PROJECT_CONSUL_SERVICE_ACCOUNT_KEY | base64 -d)
nohup terraform apply -auto-approve 2>&1 | tee nohup.out &

#### Provisoning AWS ElastiCache

In [ ]:
#aws elasticache
echo "Provisoning AWS ElastiCache..."
cd $MAIN_DIR/terraform/cache-services
terraform init
AWS_CONSUL_IP=$(terraform output -state /root/terraform/aws-consul-primary/terraform.tfstate aws_consul_public_ip)
cat <<EOF > $MAIN_DIR/terraform/cache-services/terraform.tfvars
consul_http_addr="http://127.0.0.1:8500"
EOF
nohup terraform apply -target=aws_elasticache_cluster.redis -auto-approve 2>&1 | tee nohup.out &

#### Provisoning Azure PostgreSQL

In [ ]:
#azure postgreSQL
echo "Provisoning Azure PostgreSQL..."
cd $MAIN_DIR/terraform/database-services
terraform init
AZURE_CONSUL_IP=$(terraform output -state /root/terraform/azure-consul-secondary/terraform.tfstate azure_consul_public_ip)
cat <<EOF > $MAIN_DIR/terraform/database-services/terraform.tfvars
consul_http_addr="http://127.0.0.1:8500"
EOF
nohup terraform apply -target=azurerm_postgresql_server.postgres -auto-approve 2>&1 | tee nohup.out &

## Clean Up

### Remove Cloud Assets

### Azure

Remove Azure Resource Group for Packer

In [ ]:
az group delete -y --no-wait -n ${POC_NAME}-packer

## Terraform Destroy

In [ ]:
terraform -chdir=terraform/infra plan -destroy 

In [ ]:
terraform -chdir=terraform/infra destroy -auto-approve

### Remove Assets

In [ ]:
pushd $MAIN_DIR
rm -rf $GIT_REPO_DIR apps helm packer policies terraform nohup.out
popd

# Misc

### Packer

In [ ]:
The terraform code will provision cloud infrastructure in AWS, GPC, and Azure.

Your lab environment will leverage pre-built packer images.
You can inspect the image build in the code editor, and validate the images are available in AWS & Azure.

Validate the images are available in AWS & Azure

In [ ]:
az image list -g packer | jq
# aws ec2 describe-images --owners self | jq

In [ ]:
docker run --rm -d \
    --name consul \
    -p 8500:8500 \
    -e 'VAULT_DEV_ROOT_TOKEN_ID=root' \
    -e "VAULT_ADDR=http://127.0.0.1:8200" \
    hashicorp/consul